# 核心功能与使用示例

> 读取因子数据的核心模块

In [ ]:
#| default_exp __init__

In [ ]:
#| export
import requests
import pandas as pd
from typing import Iterable,Union
from functools import wraps,reduce
import json
import datetime
from tenacity import retry, stop_after_attempt,wait_fixed

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev.config import get_config

In [ ]:
#| export
try:
    response = requests.get("https://pypi.org/pypi/factor-reader/json", timeout=2)
    latest_version = response.json()["info"]["version"]
    now_version=get_config(cfg_name='settings.ini')['version']
    if latest_version!=now_version:
        logger.warning(f'''您当前使用的是{now_version}，最新版本为{latest_version}
              建议您使用`pip install factor_reader --upgrade`
              或`pip install -i https://pypi.tuna.tsinghua.edu.cn/simple factor_reader --upgrade`命令进行更新
              ''')
except Exception:
    try:
        response = requests.get("https://pypi.org/pypi/factor-reader/json", timeout=2)
        latest_version = response.json()["info"]["version"]
        now_version=__version__
        if latest_version!=now_version:
            logger.warning(f'''您当前使用的是{now_version}，最新版本为{latest_version}
              建议您使用`pip install factor_reader --upgrade`
              或`pip install -i https://pypi.tuna.tsinghua.edu.cn/simple factor_reader --upgrade`命令进行更新
              ''')
    except Exception:
        ...

In [ ]:
# | export
class TokenUnavailableError(PermissionError):
    def __init__(self,error_info):
        super().__init__(self) 
        self.error_info=error_info
        
    def __str__(self):
        return self.error_info


def do_on_dfs_class(func):
    """一个装饰器，使类的方法的参数可以输入为列表或元组，返回值为分别对各个参数执行此函数后依次得到的结果"""

    @wraps(func)
    def wrapper(self, df=None, *args, **kwargs):
        if isinstance(df, list) or isinstance(df, tuple):
            dfs = [func(self, i, *args, **kwargs) for num, i in enumerate(df)]
            return dfs
        else:
            return func(self, df, *args, **kwargs)

    return wrapper

def concat_nosqls(func):
    @wraps(func)
    def wrapper(self,fac_key,*args,**kwargs):
        dfs=func(self,fac_key,*args,**kwargs)
        def double_index(res,key):
            res=res.reset_index().assign(name=self._deal_key_name(key)[1])
            res=res.set_index(['name','date'])
            return res
        if (isinstance(dfs, list) or isinstance(dfs, tuple)) and ("date" not in dfs[0].columns):
            dfs=[double_index(i,self._deal_key_name(key)[1]) for i,key in zip(dfs,fac_key)]
            dfs=pd.concat(dfs)
        elif (isinstance(dfs, list) or isinstance(dfs, tuple)) and ("date" in dfs[0].columns):
            dfs = reduce(
                lambda x, y: pd.merge(x, y, on=["date", "code"], how="outer"), dfs
            )
        elif "date" not in dfs.columns:
            dfs=double_index(dfs,fac_key)
        return dfs
    return wrapper



In [ ]:
# | export
class FactorReader():
    def __init__(self,token:str) -> None:
        self.token=token
        self.host='http://43.143.223.158:1837/'
        infos=self.show_all_factors_information()
        self.keys=list(infos.数据键名)
        self.names=list(infos.因子名称)
        self.keys_names = {k: v for k, v in zip(self.keys, self.names)}
        self.names_keys = {k: v for k, v in zip(self.names, self.keys)}
        
    def _deal_key_name(self,input_key):
        names_related = [i for i in self.names if input_key in i]
        if input_key in self.keys:
            fac_key=input_key
            fac_name=self.keys_names[fac_key]
        elif len(names_related) > 0:
            fac_name = names_related[0]
            fac_key = self.names_keys[fac_name]
        else:
            raise ValueError(
                "输入的fac_key参数有误，请输入因子名称或因子键名，可通过`factor_reader.show_all_factors_information`函数来查看可用的因子名称和键名"
            )
        return fac_key,fac_name
    
    @retry(stop=stop_after_attempt(100),wait=wait_fixed(3))
    def show_all_factors_information(self) -> pd.DataFrame:
        url=self.host+self.token+'/info'
        df=requests.get(url).text
        if df=='您的token暂不可用，请检查核验后重试。或联系方正金工团队开通可用token~':
            raise TokenUnavailableError('您的token暂不可用，请检查核验后重试。或联系方正金工团队开通可用token~')
        df=pd.read_json(json.loads(df))
        return df
    
    @concat_nosqls
    @retry(stop=stop_after_attempt(100),wait=wait_fixed(3))
    @do_on_dfs_class
    def read_factor(
        self,
        fac_key: str,
        trade_date: Union[int,str,datetime.datetime] = None,
        start_date: Union[int,str,datetime.datetime] = None,
        end_date: Union[int,str,datetime.datetime] = None,
        sql_like: bool = False,
    ) -> pd.DataFrame:
        """通过表名，读取因子数据

        Parameters
        ----------
        fac_key : str
            表的名称或因子的名称
        trade_date : Union[int,str,datetime.datetime], optional
            读取单日因子值，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13')，指定此参数时，start_date和end_date两个参数将失效, by default None
        start_date : Union[int,str,datetime.datetime], optional
            读取因子值的起始日期，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13'), by default None
        end_date : Union[int,str,datetime.datetime], optional
            读取因子值的终止日期，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13'), by default None
        sql_like : bool, optional
            返回的数据为形如sql中的长表，包括日期、股票代码、因子值三列, by default False

        Returns
        -------
        pd.DataFrame
            因子值，index为每天的日期，columns为股票代码，values为因子值
        """
        def date_ok(x):
            if isinstance(x,int):
                date=pd.Timestamp(str(x))
            elif isinstance(x,float):
                date=pd.Timestamp(str(int(x)))
            else:
                date=pd.Timestamp(x)
            date=datetime.datetime.strftime(date,'%Y%m%d')
            return date
        
        '''名称部分处理'''
        fac_key,fac_name=self._deal_key_name(fac_key)
        print(f"正在读取{fac_name}的数据")
        
        '''日期部分处理'''
        if trade_date is not None:
            trade_date=date_ok(trade_date)
            start_date='None'
            end_date='None'
        elif (start_date is not None) and (end_date is not None):
            start_date=date_ok(start_date)
            end_date=date_ok(end_date)
            trade_date='None'
        elif start_date is not None:
            start_date=date_ok(start_date)
            trade_date='None'
            end_date='None'
        elif end_date is not None:
            end_date=date_ok(end_date)
            trade_date='None'
            start_date='None'
        else:
            trade_date='None'
            start_date='None'
            end_date='None'
            
        url=self.host+self.token+'/'+fac_key+'/'+trade_date+'/'+start_date+'/'+end_date+'/'+str(int(sql_like))
        res=requests.get(url).text
        self.res=res
        if res=='您的token暂不可用，请检查核验后重试。或联系方正金工团队开通可用token~':
            raise TokenUnavailableError('您的token暂不可用，请检查核验后重试。或联系方正金工团队开通可用token~')
        res=pd.read_json(json.loads(res))
        if 'timestamp' in res.columns:
            res=res.drop(columns=['timestamp'])
        if not sql_like:
            res=res.pivot(index='date',columns='code',values='fac')
        else:
            res=res.rename(columns={'fac':self.keys_names[fac_key]})
        
        return res

In [ ]:
show_doc(FactorReader.show_all_factors_information)

---

### FactorReader.show_all_factors_information

>      FactorReader.show_all_factors_information ()

In [ ]:
show_doc(FactorReader.read_factor)

---

### FactorReader.read_factor

>      FactorReader.read_factor (fac_key:str,
>                                trade_date:Union[int,str,datetime.datetime]=Non
>                                e, start_date:Union[int,str,datetime.datetime]=
>                                None,
>                                end_date:Union[int,str,datetime.datetime]=None,
>                                sql_like:bool=False)

通过表名，读取因子数据

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| fac_key | str |  | 表的名称或因子的名称 |
| trade_date | Union | None | 读取单日因子值，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13')，指定此参数时，start_date和end_date两个参数将失效, by default None |
| start_date | Union | None | 读取因子值的起始日期，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13'), by default None |
| end_date | Union | None | 读取因子值的终止日期，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13'), by default None |
| sql_like | bool | False | 返回的数据为形如sql中的长表，包括日期、股票代码、因子值三列, by default False |
| **Returns** | **DataFrame** |  | **因子值，index为每天的日期，columns为股票代码，values为因子值** |

# 使用示例

In [ ]:
token='pypi' #|hide_line

## 第一步：实例化一个FactorReader对象，输入token

In [ ]:
fr=FactorReader(token=token) # 初始化一个FactorReader对象，输入token，连接因子数据库

## 第二步：查看当前因子数据库中有哪些因子（此步骤可选）

In [ ]:
fr.show_all_factors_information() # 展示所有可读取因子的相关信息

,数据键名,因子名称,报告题目,微信链接
0,factor1,适度冒险因子,成交量激增时刻蕴含的alpha信息——多因子选股系列研究之一,https://mp.weixin.qq.com/s/2pObrtp3V0dv50MFGg_fhw
1,factor2,完整潮汐因子,个股成交量的潮汐变化及“潮汐”因子构建——多因子选股系列研究之二,https://mp.weixin.qq.com/s/_2xWXM8iyNzYDYolT9vVsw
2,factor3,勇攀高峰因子,个股波动率的变动及”勇攀高峰“因子构建——多因子选股系列研究之三,https://mp.weixin.qq.com/s/-IxH3n-uR0BwOIbOyzqPrw
3,factor4,球队硬币因子,个股动量效应的识别及“球队硬币”因子构建——多因子选股系列研究之四,https://mp.weixin.qq.com/s/JCv8qziZ_ZfTbBE6l8cKOQ
4,factor5,云开雾散因子,波动率的波动率与投资者模糊性厌恶——多因子选股系列研究之五,https://mp.weixin.qq.com/s/vX4I9SpKRF3_HKQOGhtZcQ
5,factor6,飞蛾扑火因子,个股股价跳跃及其对振幅因子的改进——多因子选股系列研究之六,https://mp.weixin.qq.com/s/V8r1Xbz5J0A9D5J_9GD_IA
6,factor8,草木皆兵因子,显著效应、极端收益扭曲决策权重和“草木皆兵”因子——多因子选股系列研究之八,https://mp.weixin.qq.com/s/hdtzQaCF2h6ZjZd-0HAftQ
7,factor9,水中行舟因子,个股成交额的市场跟随性与“水中行舟”因子——多因子选股系列研究之九,https://mp.weixin.qq.com/s/Zv6XX8ddUvBCtG-4LM1MKQ


## 第三步：读取因子数据

### ①读取单日单个因子的数据

In [ ]:
fr.read_factor('球队硬币',trade_date=20230106) # 读取球队硬币因子在2023年1月13日的因子数据

正在读取球队硬币因子的数据


,code,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,871970.BJ,871981.BJ,872374.BJ,872808.BJ,872925.BJ,873122.BJ,873169.BJ,873223.BJ,873339.BJ,873527.BJ
name,date,,,,,,,,,,,,,,,,,,,,,
球队硬币因子,2023-01-06,-3.45854,-0.674148,0.220243,-0.950252,8.786346,0.500339,-1.828122,0.267091,-0.781898,2.299623,...,0.104238,-1.345124,-1.070172,-3.192394,-1.628848,0.60037,-2.126244,-0.90467,-0.537109,-1.108383


In [ ]:
fr.read_factor('factor4',trade_date='20230106') # 读取方正金工多因子系列第4篇的因子在2023年1月13日的因子数据

正在读取球队硬币因子的数据


,code,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,871970.BJ,871981.BJ,872374.BJ,872808.BJ,872925.BJ,873122.BJ,873169.BJ,873223.BJ,873339.BJ,873527.BJ
name,date,,,,,,,,,,,,,,,,,,,,,
球队硬币因子,2023-01-06,-3.45854,-0.674148,0.220243,-0.950252,8.786346,0.500339,-1.828122,0.267091,-0.781898,2.299623,...,0.104238,-1.345124,-1.070172,-3.192394,-1.628848,0.60037,-2.126244,-0.90467,-0.537109,-1.108383


### ②读取多个交易日单个因子的数据

In [ ]:
fr.read_factor('球队硬币',start_date='2023-01-01',end_date=pd.Timestamp('20230106')) # 读取球队硬币因子从2023年1月1日至2023年1月13日的因子数据

正在读取球队硬币因子的数据


code               000001.SZ  000002.SZ  000004.SZ  000005.SZ  000006.SZ  \
name   date                                                                
球队硬币因子 2023-01-03  -2.358642  -1.345033   1.162492   0.007223   8.697314   
       2023-01-04  -3.250196  -0.612765   1.104373  -0.604997   8.853588   
       2023-01-05  -3.612259  -0.353330   0.683165  -0.712644   8.803313   
       2023-01-06  -3.458540  -0.674148   0.220243  -0.950252   8.786346   

code               000007.SZ  000008.SZ  000009.SZ  000010.SZ  000011.SZ  ...  \
name   date                                                               ...   
球队硬币因子 2023-01-03   1.189680  -1.702174  -1.052431  -0.854015   3.398331  ...   
       2023-01-04   0.434524  -2.027665  -1.078369  -1.319052   2.476316  ...   
       2023-01-05   0.304470  -1.978654  -0.557577  -1.869844   2.610491  ...   
       2023-01-06   0.500339  -1.828122   0.267091  -0.781898   2.299623  ...   

code               871970.BJ  871981.BJ  872374.BJ  872808.BJ  872925.BJ  \
name   date                                                                
球队硬币因子 2023-01-03  -0.103756  -1.137786  -1.021316  -3.253337  -1.775420   
       2023-01-04  -0.088794  -1.116035  -1.024373  -3.126951  -1.740175   
       2023-01-05   0.010475  -1.115315  -1.121056  -3.268384  -1.753120   
       2023-01-06   0.104238  -1.345124  -1.070172  -3.192394  -1.628848   

code               873122.BJ  873169.BJ  873223.BJ  873339.BJ  873527.BJ  
name   date                                                               
球队硬币因子 2023-01-03   2.468078  -1.900470  -1.004885  -0.407954  -0.799920  
       2023-01-04   2.741972  -2.164878  -0.959739  -0.397286  -0.845371  
       2023-01-05   1.020633  -2.132997  -0.893447  -0.449713  -0.817057  
       2023-01-06   0.600370  -2.126244  -0.904670  -0.537109  -1.108383  

[4 rows x 5017 columns]

In [ ]:
fr.read_factor('factor4',start_date=20230101,end_date=20230106,sql_like=True) # 以sql的形式，读取方正金工多因子系列第4篇的因子从2023年1月1日至2023年1月13日的因子数据

正在读取球队硬币因子的数据


,code,date,球队硬币因子
0,000001.SZ,2023-01-03,-2.358642
1,000002.SZ,2023-01-03,-1.345033
2,000004.SZ,2023-01-03,1.162492
3,000005.SZ,2023-01-03,0.007223
4,000006.SZ,2023-01-03,8.697314
...,...,...,...
20045,873122.BJ,2023-01-06,0.600370
20046,873169.BJ,2023-01-06,-2.126244
20047,873223.BJ,2023-01-06,-0.904670
20048,873339.BJ,2023-01-06,-0.537109


### ③读取多个交易日多个因子的因子数据

In [ ]:
fr.read_factor(['球队硬币','云开雾散'],start_date='20230101',end_date='20230106') # 分别读取球队硬币因子和云开雾散因子从2023年1月1日至2023年1月13日的因子数据

正在读取球队硬币因子的数据
正在读取云开雾散因子的数据


code               000001.SZ  000002.SZ  000004.SZ  000005.SZ  000006.SZ  \
name   date                                                                
球队硬币因子 2023-01-03  -2.358642  -1.345033   1.162492   0.007223   8.697314   
       2023-01-04  -3.250196  -0.612765   1.104373  -0.604997   8.853588   
       2023-01-05  -3.612259  -0.353330   0.683165  -0.712644   8.803313   
       2023-01-06  -3.458540  -0.674148   0.220243  -0.950252   8.786346   
云开雾散因子 2023-01-03  -0.981326  -0.944349  -0.997003   4.198288   3.824620   
       2023-01-04  -0.874760  -0.565374  -1.418436  -2.256764   3.836005   
       2023-01-05  -0.254081  -0.614854  -1.407234  -2.364404   3.869275   
       2023-01-06  -0.291150  -0.524840  -1.476704  -3.120565   3.906959   

code               000007.SZ  000008.SZ  000009.SZ  000010.SZ  000011.SZ  ...  \
name   date                                                               ...   
球队硬币因子 2023-01-03   1.189680  -1.702174  -1.052431  -0.854015   3.398331  ...   
       2023-01-04   0.434524  -2.027665  -1.078369  -1.319052   2.476316  ...   
       2023-01-05   0.304470  -1.978654  -0.557577  -1.869844   2.610491  ...   
       2023-01-06   0.500339  -1.828122   0.267091  -0.781898   2.299623  ...   
云开雾散因子 2023-01-03  -0.305261  -2.440184   1.778184  -1.547923   2.822911  ...   
       2023-01-04  -0.601352  -1.189656   1.456942  -1.902164   2.427931  ...   
       2023-01-05  -1.362426  -1.225968   1.424493  -1.910795   2.395279  ...   
       2023-01-06  -1.295554  -1.253691  -0.280816   0.352601   2.508948  ...   

code               301398.SZ  600038.SH  600459.SH  600663.SH  600729.SH  \
name   date                                                                
球队硬币因子 2023-01-03        NaN        NaN        NaN        NaN        NaN   
       2023-01-04        NaN        NaN        NaN        NaN        NaN   
       2023-01-05        NaN        NaN        NaN        NaN        NaN   
       2023-01-06        NaN        NaN        NaN        NaN        NaN   
云开雾散因子 2023-01-03   2.320428  -0.672918  -0.150167   1.191432   0.075872   
       2023-01-04   2.093086  -0.477727  -0.157869   1.731492  -0.314115   
       2023-01-05   2.492977  -0.415604  -0.156190   1.573648   0.032488   
       2023-01-06   2.564407  -0.250353   0.183853   1.403672  -0.088864   

code               603029.SH  688141.SH  688147.SH  688172.SH  688498.SH  
name   date                                                               
球队硬币因子 2023-01-03        NaN        NaN        NaN        NaN        NaN  
       2023-01-04        NaN        NaN        NaN        NaN        NaN  
       2023-01-05        NaN        NaN        NaN        NaN        NaN  
       2023-01-06        NaN        NaN        NaN        NaN        NaN  
云开雾散因子 2023-01-03  10.216414        NaN        NaN   1.769576        NaN  
       2023-01-04  10.224871        NaN        NaN   1.559961   1.156921  
       2023-01-05   9.917723        NaN        NaN   1.374394   1.120350  
       2023-01-06  10.332152  -0.121816   0.644811   1.282682   0.950078  

[8 rows x 5033 columns]

In [ ]:
fr.read_factor(['球队硬币','云开雾散'],trade_date=20230106)

正在读取球队硬币因子的数据
正在读取云开雾散因子的数据


,code,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,301398.SZ,600038.SH,600459.SH,600663.SH,600729.SH,603029.SH,688141.SH,688147.SH,688172.SH,688498.SH
name,date,,,,,,,,,,,,,,,,,,,,,
球队硬币因子,2023-01-06,-3.45854,-0.674148,0.220243,-0.950252,8.786346,0.500339,-1.828122,0.267091,-0.781898,2.299623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
云开雾散因子,2023-01-06,-0.29115,-0.524840,-1.476704,-3.120565,3.906959,-1.295554,-1.253691,-0.280816,0.352601,2.508948,...,2.564407,-0.250353,0.183853,1.403672,-0.088864,10.332152,-0.121816,0.644811,1.282682,0.950078


In [ ]:
fr.read_factor(['球队硬币','云开雾散','飞蛾扑火'],start_date=pd.Timestamp('20230101'),end_date=20230106,sql_like=True) # 以sql的形式，分别读取球队硬币因子、云开雾散因子、飞蛾扑火因子从2023年1月1日至2023年1月13日的因子数据

正在读取球队硬币因子的数据
正在读取云开雾散因子的数据
正在读取飞蛾扑火因子的数据


,code,date,球队硬币因子,云开雾散因子,飞蛾扑火因子
0,000001.SZ,2023-01-03,-2.358642,-0.981326,-1.211546
1,000002.SZ,2023-01-03,-1.345033,-0.944349,-0.747704
2,000004.SZ,2023-01-03,1.162492,-0.997003,-1.218871
3,000005.SZ,2023-01-03,0.007223,4.198288,-0.089553
4,000006.SZ,2023-01-03,8.697314,3.824620,6.631622
...,...,...,...,...,...
20114,688498.SH,2023-01-06,NaN,0.950078,5.589459
20115,000153.SZ,2023-01-04,NaN,NaN,3.513004
20116,000153.SZ,2023-01-05,NaN,NaN,4.299205
20117,000153.SZ,2023-01-06,NaN,NaN,2.936270


In [ ]:
fr.read_factor(['球队硬币','云开雾散','飞蛾扑火'],trade_date=20230106,sql_like=True) # 以sql的形式，分别读取球队硬币因子、云开雾散因子、飞蛾扑火因子2023年1月13日的因子数据

正在读取球队硬币因子的数据
正在读取云开雾散因子的数据
正在读取飞蛾扑火因子的数据


,code,date,球队硬币因子,云开雾散因子,飞蛾扑火因子
0,000001.SZ,2023-01-06,-3.458540,-0.291150,-0.952147
1,000002.SZ,2023-01-06,-0.674148,-0.524840,-0.743429
2,000004.SZ,2023-01-06,0.220243,-1.476704,-1.224969
3,000005.SZ,2023-01-06,-0.950252,-3.120565,-0.757643
4,000006.SZ,2023-01-06,8.786346,3.906959,6.767050
...,...,...,...,...,...
5029,688147.SH,2023-01-06,NaN,0.644811,NaN
5030,688172.SH,2023-01-06,NaN,1.282682,0.124283
5031,688498.SH,2023-01-06,NaN,0.950078,5.589459
5032,000153.SZ,2023-01-06,NaN,NaN,2.936270


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()